<a href="https://colab.research.google.com/github/neel0086/deep_learning/blob/main/CustomerChurn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [9]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['customer_churn.csv']))
df.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
6654,1925-TIBLE,Female,0,Yes,No,7,Yes,Yes,Fiber optic,No,...,No,No,Yes,No,Month-to-month,Yes,Electronic check,86.50,582.5,Yes
2913,6203-HBZPA,Male,0,No,No,52,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),63.90,3334.95,No
2336,3211-ILJTT,Male,0,Yes,No,17,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),70.40,1214.05,Yes
4730,7813-ZGGAW,Male,1,No,No,31,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,No,Month-to-month,Yes,Bank transfer (automatic),96.60,2877.95,No
4462,6087-MVHJH,Female,0,No,No,16,Yes,Yes,Fiber optic,No,...,Yes,No,No,Yes,Month-to-month,Yes,Electronic check,88.45,1422.1,Yes
